# Data Wrangling

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt

## Import files

In [2]:
#Import csv file
tae = pd.read_csv('twitter-archive-enhanced.csv')

## Dataframes

### Describe data

In [3]:
tae.head()
# tae.tail()
# tae.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
tae.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [5]:
tae.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

### Handle missing data

In [6]:
tae.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [7]:
tae.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [8]:
# drops rows where any column has na
tae.dropna(axis=0) # drop na rows
tae.dropna(axis=1) # drop na columns

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,5,10,None,None,None,None,None
2352,666044226329800704,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,6,10,a,None,None,None,None
2353,666033412701032449,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,9,10,a,None,None,None,None
2354,666029285002620928,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,7,10,a,None,None,None,None


In [9]:
# replaces na with value
tae.fillna('hello')
tae.fillna(0)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,0.0,0.0,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0.0,0.0,0,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,0.0,0.0,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0.0,0.0,0,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,0.0,0.0,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0.0,0.0,0,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,0.0,0.0,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,0.0,0.0,0,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,0.0,0.0,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,0.0,0.0,0,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,0.0,0.0,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,0.0,0.0,0,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,0.0,0.0,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,0.0,0.0,0,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,0.0,0.0,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,0.0,0.0,0,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,0.0,0.0,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,0.0,0.0,0,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


In [10]:
tae.fillna(method='ffill', axis=0) # use previous known value in column
tae.fillna(method='ffill', axis=1) # use previous known value in row
tae.fillna(method='backfill', axis=0) # use next known value in column
tae.fillna(method='backfill', axis=1) # use next known value in row
tae.interpolate(method='linear', axis=0) # use previous known value in column

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,6.670655e+17,4.196984e+09,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,6.675484e+17,4.296832e+09,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,6.670655e+17,4.196984e+09,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,6.675484e+17,4.296832e+09,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,6.670655e+17,4.196984e+09,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,6.675484e+17,4.296832e+09,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,6.670655e+17,4.196984e+09,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,6.675484e+17,4.296832e+09,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


### Handle duplicates

In [11]:
tae.drop_duplicates()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


### Change data types

In [12]:
tae.tweet_id = tae.tweet_id.astype(float)

### Create dataframe

In [13]:
# create dataframe from list (column names need to match)
# df = pd.DataFrame(list, columns=['xyz', 'xyz'])

### Sort values

In [14]:
tae.sort_values('tweet_id', ascending=True)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2355,6.660209e+17,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None
2354,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2353,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2352,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2351,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
1,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [15]:
tae.sort_index()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


### Access rows

In [16]:
tae.iloc[1224:1225]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1224,7.142141e+17,NaN,NaN,2016-03-27 22:14:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Easter from the squad! 🐇🐶 13/10 for all ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714214115...,13,10,None,None,None,None,None


In [17]:
# difference between loc (index) and iloc (new "index")
tae.loc[0:4]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [18]:
tae.index

RangeIndex(start=0, stop=2356, step=1)

### Create rows

In [19]:
# new_row = pd.DataFrame({'tweet_id':1, 'rating_numerator':15, 'rating':3, 'rating_calc':4}, index=['11'])

In [20]:
# df.append(new_row)

### Drop rows

In [21]:
tae.drop([9], axis=0, inplace=True)

### Rename rows / reset index

In [22]:
# moves index to column
longer_tae = tae.reset_index()
longer_tae = tae.reset_index(drop=True, inplace=True)
tae.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [23]:
tae.rename(index={1:'hallo'}, inplace=True)
tae.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [24]:
longer_tae = tae.set_index('tweet_id')
longer_tae

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
tweet_id,,,,,,,,,,,,,,,,
8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None


### Access columns

In [25]:
tae[['tweet_id', 'timestamp']]

,tweet_id,timestamp
0,8.924206e+17,2017-08-01 16:23:56 +0000
hallo,8.921774e+17,2017-08-01 00:17:27 +0000
2,8.918152e+17,2017-07-31 00:18:03 +0000
3,8.916896e+17,2017-07-30 15:58:51 +0000
4,8.913276e+17,2017-07-29 16:00:24 +0000
...,...,...
2350,6.660492e+17,2015-11-16 00:24:50 +0000
2351,6.660442e+17,2015-11-16 00:04:52 +0000
2352,6.660334e+17,2015-11-15 23:21:54 +0000
2353,6.660293e+17,2015-11-15 23:05:30 +0000


In [26]:
tae.iloc[:, [1,3,5]]

,in_reply_to_status_id,timestamp,text
0,NaN,2017-08-01 16:23:56 +0000,This is Phineas. He's a mystical boy. Only eve...
hallo,NaN,2017-08-01 00:17:27 +0000,This is Tilly. She's just checking pup on you....
2,NaN,2017-07-31 00:18:03 +0000,This is Archie. He is a rare Norwegian Pouncin...
3,NaN,2017-07-30 15:58:51 +0000,This is Darla. She commenced a snooze mid meal...
4,NaN,2017-07-29 16:00:24 +0000,This is Franklin. He would like you to stop ca...
...,...,...,...
2350,NaN,2015-11-16 00:24:50 +0000,Here we have a 1949 1st generation vulpix. Enj...
2351,NaN,2015-11-16 00:04:52 +0000,This is a purebred Piers Morgan. Loves to Netf...
2352,NaN,2015-11-15 23:21:54 +0000,Here is a very happy pup. Big fan of well-main...
2353,NaN,2015-11-15 23:05:30 +0000,This is a western brown Mitsubishi terrier. Up...


In [27]:
tae.loc[:, 'tweet_id':'timestamp']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000
...,...,...,...,...
2350,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000
2351,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000
2352,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000
2353,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000


In [28]:
columns = tae.columns
columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

### Drop columns

In [29]:
tae.drop(columns=['tweet_id']) # inplace=True
tae.drop(['rating_numerator'], axis=1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,10,Phineas,None,None,None,None
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,10,Tilly,None,None,None,None
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,10,Archie,None,None,None,None
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,10,Darla,None,None,None,None
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,10,None,None,None,None,None
2351,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,10,a,None,None,None,None
2352,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,10,a,None,None,None,None
2353,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,10,a,None,None,None,None


### Create columns

In [30]:
tae['rating'] = tae['rating_denominator'] + tae['rating_numerator']

In [31]:
tae.rating_sum = tae.rating_numerator/2

<ipython-input-31-0e691c394311>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tae.rating_sum = tae.rating_numerator/2


In [32]:
# df.insert(2, 'rate', [3,3,3,3,3,3,3,3,3,3,3])

### Rename columns

In [33]:
tae = tae.rename(columns={'text':'test'})
# tae.rename(columns={'timestamp':'time'}, inplace=True)
# tae = tae.rename(columns={'rating_numerator':'rating', 'text':'test'})
tae.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,23
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,23
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,22
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,23
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,22


### Query data and build subsets

In [34]:
tae[['tweet_id', 'timestamp']].iloc[1224:1228]

,tweet_id,timestamp
1224,7.141414e+17,2016-03-27 17:25:54 +0000
1225,7.139195e+17,2016-03-27 02:43:58 +0000
1226,7.139099e+17,2016-03-27 02:05:49 +0000
1227,7.139006e+17,2016-03-27 01:29:02 +0000


In [35]:
# logical condition
tae.loc[tae['tweet_id'] > 10, ['timestamp', 'text']]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['text'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
# build new dataframe
data = tae[['tweet_id', 'timestamp']].iloc[1224:1228]

In [36]:
# query dataframe
tae.query('tweet_id == 714214115368108032')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
1223,7.142141e+17,NaN,NaN,2016-03-27 22:14:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Easter from the squad! 🐇🐶 13/10 for all ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714214115...,13,10,None,None,None,None,None,23


In [37]:
tae.query('tweet_id <= 884162670584377345')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
42,8.841627e+17,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,None,None,None,22
43,8.838381e+17,NaN,NaN,2017-07-09 00:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Noah. He can't believe someone made th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/883838122...,12,10,Noah,None,None,None,None,22
44,8.834828e+17,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bella. She hopes her smile made you sm...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/883482846...,5,10,Bella,None,None,None,None,15
45,8.833607e+17,NaN,NaN,2017-07-07 16:22:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Grizzwald. He may be the floofiest floofe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/883360690...,13,10,Grizzwald,None,floofer,None,None,23
46,8.831178e+17,NaN,NaN,2017-07-07 00:17:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please only send dogs. We don't rate mechanics...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/883117836...,13,10,None,None,None,None,None,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None,15
2351,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None,16
2352,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None,19
2353,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None,17


In [38]:
tae.query("name == 'Phineas'")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,23
2103,6.706684e+17,NaN,NaN,2015-11-28 18:19:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a magical dog. Only appe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670668383...,10,10,Phineas,None,None,None,None,20


In [39]:
tae[tae.tweet_id > 890000070584377345]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,23
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,23
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,22
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,23
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,22
5,8.910880e+17,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,23
6,8.909719e+17,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,23
7,8.907292e+17,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,23
8,8.906092e+17,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,23
9,8.900066e+17,NaN,NaN,2017-07-26 00:31:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Koda. He is a South Australian decksha...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890006608...,13,10,Koda,None,None,None,None,23


In [40]:
# select and order top entries
tae.nlargest(10, 'tweet_id')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
0,8.924206e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,23
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,23
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,22
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,23
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,22
5,8.910880e+17,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,23
6,8.909719e+17,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,23
7,8.907292e+17,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,23
8,8.906092e+17,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,23
9,8.900066e+17,NaN,NaN,2017-07-26 00:31:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Koda. He is a South Australian decksha...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890006608...,13,10,Koda,None,None,None,None,23


In [41]:
# select and order bottom entries
tae.nsmallest(10, 'tweet_id')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
2354,6.660209e+17,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None,18
2353,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None,17
2352,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None,19
2351,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None,16
2350,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None,15
2349,6.660508e+17,NaN,NaN,2015-11-16 00:30:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a truly beautiful English Wilson Staff...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758...,10,10,a,None,None,None,None,20
2348,6.660519e+17,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None,12
2347,6.660555e+17,NaN,NaN,2015-11-16 00:49:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a Siberian heavily armored polar bear ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666055525...,10,10,a,None,None,None,None,20
2346,6.660571e+17,NaN,NaN,2015-11-16 00:55:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",My oh my. This is a rare blond Canadian terrie...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666057090...,9,10,a,None,None,None,None,19
2345,6.660586e+17,NaN,NaN,2015-11-16 01:01:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is the Rand Paul of retrievers folks! He'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666058600...,8,10,the,None,None,None,None,18


### Reshaping data

#### Melt

In [42]:
short_df = tae.query('tweet_id >= 890000070584377345')[['tweet_id', 'rating_numerator']]
short_df

,tweet_id,rating_numerator
0,8.924206e+17,13
hallo,8.921774e+17,13
2,8.918152e+17,12
3,8.916896e+17,13
4,8.913276e+17,12
5,8.910880e+17,13
6,8.909719e+17,13
7,8.907292e+17,13
8,8.906092e+17,13
9,8.900066e+17,13


In [43]:
melt_df = pd.melt(short_df)
melt_df

,variable,value
0,tweet_id,8.924206e+17
1,tweet_id,8.921774e+17
2,tweet_id,8.918152e+17
3,tweet_id,8.916896e+17
4,tweet_id,8.913276e+17
5,tweet_id,8.910880e+17
6,tweet_id,8.909719e+17
7,tweet_id,8.907292e+17
8,tweet_id,8.906092e+17
9,tweet_id,8.900066e+17


#### Concat

In [44]:
short_df_1 = short_df.iloc[0:5]
short_df_1

,tweet_id,rating_numerator
0,8.924206e+17,13
hallo,8.921774e+17,13
2,8.918152e+17,12
3,8.916896e+17,13
4,8.913276e+17,12


In [45]:
short_df_2 = short_df.iloc[5:11]
short_df_2

,tweet_id,rating_numerator
5,8.910880e+17,13
6,8.909719e+17,13
7,8.907292e+17,13
8,8.906092e+17,13
9,8.900066e+17,13


In [46]:
concat_df = pd.concat([short_df_2, short_df_1])
concat_df

,tweet_id,rating_numerator
5,8.910880e+17,13
6,8.909719e+17,13
7,8.907292e+17,13
8,8.906092e+17,13
9,8.900066e+17,13
0,8.924206e+17,13
hallo,8.921774e+17,13
2,8.918152e+17,12
3,8.916896e+17,13
4,8.913276e+17,12


In [47]:
small_df_1 = short_df['tweet_id']
small_df_2 = short_df['rating_numerator']

In [48]:
concat_df_sideways = pd.concat([small_df_2, small_df_1], axis=1)
concat_df_sideways

,rating_numerator,tweet_id
0,13,8.924206e+17
hallo,13,8.921774e+17
2,12,8.918152e+17
3,13,8.916896e+17
4,12,8.913276e+17
5,13,8.910880e+17
6,13,8.909719e+17
7,13,8.907292e+17
8,13,8.906092e+17
9,13,8.900066e+17


#### Merge

In [49]:
pd.merge(short_df_1, short_df_2, how='outer')

,tweet_id,rating_numerator
0,8.924206e+17,13
1,8.921774e+17,13
2,8.918152e+17,12
3,8.916896e+17,13
4,8.913276e+17,12
5,8.910880e+17,13
6,8.909719e+17,13
7,8.907292e+17,13
8,8.906092e+17,13
9,8.900066e+17,13


In [50]:
pd.merge(short_df_1, short_df_2, how='left', on='tweet_id')

,tweet_id,rating_numerator_x,rating_numerator_y
0,8.924206e+17,13,NaN
1,8.921774e+17,13,NaN
2,8.918152e+17,12,NaN
3,8.916896e+17,13,NaN
4,8.913276e+17,12,NaN


In [51]:
pd.merge(short_df_1, short_df_2, how='right', on='tweet_id')

,tweet_id,rating_numerator_x,rating_numerator_y
0,8.910880e+17,NaN,13
1,8.909719e+17,NaN,13
2,8.907292e+17,NaN,13
3,8.906092e+17,NaN,13
4,8.900066e+17,NaN,13


In [52]:
pd.merge(short_df_1, short_df_2, how='outer', on='tweet_id')

,tweet_id,rating_numerator_x,rating_numerator_y
0,8.924206e+17,13.0,NaN
1,8.921774e+17,13.0,NaN
2,8.918152e+17,12.0,NaN
3,8.916896e+17,13.0,NaN
4,8.913276e+17,12.0,NaN
5,8.910880e+17,NaN,13.0
6,8.909719e+17,NaN,13.0
7,8.907292e+17,NaN,13.0
8,8.906092e+17,NaN,13.0
9,8.900066e+17,NaN,13.0


In [53]:
pd.merge(short_df_1, short_df_2, how='inner', on='tweet_id')

,tweet_id,rating_numerator_x,rating_numerator_y


#### Pivot

In [54]:
short_df.pivot(columns='rating_numerator', values='tweet_id')

rating_numerator,12,13
0,NaN,8.924206e+17
2,8.918152e+17,NaN
3,NaN,8.916896e+17
4,8.913276e+17,NaN
5,NaN,8.910880e+17
6,NaN,8.909719e+17
7,NaN,8.907292e+17
8,NaN,8.906092e+17
9,NaN,8.900066e+17
hallo,NaN,8.921774e+17


### Example

In [55]:
dog_columns = ['doggo', 'floofer', 'pupper', 'puppo']

for column in dog_columns:
    tae[column] = tae[column].str.replace('None','')

In [56]:
new_dog = tae[['doggo', 'floofer', 'pupper', 'puppo']]
new_dog

,doggo,floofer,pupper,puppo
0,,,,
hallo,,,,
2,,,,
3,,,,
4,,,,
...,...,...,...,...
2350,,,,
2351,,,,
2352,,,,
2353,,,,


In [57]:
new_dog['new_column'] = new_dog['doggo'] + new_dog['floofer'] + new_dog['pupper'] + new_dog['puppo']

<ipython-input-57-eecb94a07185>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dog['new_column'] = new_dog['doggo'] + new_dog['floofer'] + new_dog['pupper'] + new_dog['puppo']


In [58]:
new_dog['new_column'] = new_dog['new_column'].str.replace('doggopupper','undecided')
new_dog['new_column'] = new_dog['new_column'].str.replace('doggopuppo','undecided')
new_dog['new_column'] = new_dog['new_column'].str.replace('doggofloofer','undecided')

<ipython-input-58-bbab528c1e30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dog['new_column'] = new_dog['new_column'].str.replace('doggopupper','undecided')
<ipython-input-58-bbab528c1e30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dog['new_column'] = new_dog['new_column'].str.replace('doggopuppo','undecided')
<ipython-input-58-bbab528c1e30>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

### Describe data

In [59]:
tae.tweet_id.max()

8.924206435553362e+17

In [60]:
tae.corr()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator,rating
tweet_id,1.000000,0.940568,0.135083,0.744733,0.144644,0.049528,-0.028239,0.043952
in_reply_to_status_id,0.940568,1.000000,0.136589,NaN,NaN,0.265525,-0.109931,0.252998
in_reply_to_user_id,0.135083,0.136589,1.000000,NaN,NaN,-0.034593,-0.019973,-0.036332
retweeted_status_id,0.744733,NaN,NaN,1.000000,0.168284,0.171930,-0.037949,0.171461
retweeted_status_user_id,0.144644,NaN,NaN,0.168284,1.000000,0.016674,-0.009668,0.016539
rating_numerator,0.049528,0.265525,-0.034593,0.171930,0.016674,1.000000,0.150388,0.990039
rating_denominator,-0.028239,-0.109931,-0.019973,-0.037949,-0.009668,0.150388,1.000000,0.288085
rating,0.043952,0.252998,-0.036332,0.171461,0.016539,0.990039,0.288085,1.000000


### Group by

In [61]:
tae.groupby(['rating_numerator', 'source'])['rating_denominator'].mean()

rating_numerator  source                                                                             
0                 <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>      10.0
1                 <a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                      10.0
                  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>       9.0
2                 <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>      10.0
3                 <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>      10.0
                                                                                                         ...  
204               <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     170.0
420               <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>      10.0
666       

In [62]:
tae.groupby(['rating_numerator'])['rating_denominator'].cummax()
# / .size / .value_counts / .rank(method='dense' or 'min' or 'first' or 'pct=True') (ranks with no gaps) 
# / .cumsum (cumulative sum) / .cummax / .cummin / .cumprod

0        10
hallo    10
2        10
3        10
4        10
         ..
2350     10
2351     10
2352     11
2353     11
2354     10
Name: rating_denominator, Length: 2355, dtype: int64

In [63]:
tae.groupby('rating_denominator')['rating_numerator'].mean() 
#/ .sum / .count / .median / .quantile([0.25,0.75]) / .min / .max / .mean / .var / .std
#/ .apply(function) - apply function to each object

rating_denominator
0      960.000000
2        1.000000
7       24.000000
10      12.236707
11       8.333333
15      11.000000
16      20.000000
20       4.000000
40      44.000000
50      51.666667
70      84.000000
80      84.000000
90      99.000000
110    121.000000
120    144.000000
130    143.000000
150    165.000000
170    204.000000
Name: rating_numerator, dtype: float64

## List

In [64]:
new_list = [1, 5, 3]

### Create list (from dataframe)

In [65]:
tweet_id_list = tae.tweet_id.tolist()
tweet_id_list

[8.924206435553362e+17,
 8.921774213063434e+17,
 8.918151813780849e+17,
 8.916895572798587e+17,
 8.913275589266883e+17,
 8.910879508758979e+17,
 8.909719131739914e+17,
 8.907291814112379e+17,
 8.906091851503124e+17,
 8.900066081131725e+17,
 8.898808964798669e+17,
 8.896653883336827e+17,
 8.896388375799071e+17,
 8.895311353442099e+17,
 8.892788419816858e+17,
 8.889172381238313e+17,
 8.888049891996713e+17,
 8.885549627242783e+17,
 8.882025155730883e+17,
 8.880784344585871e+17,
 8.877052893818266e+17,
 8.875171391580938e+17,
 8.874739571039519e+17,
 8.873432170453688e+17,
 8.871013928040858e+17,
 8.869832335225446e+17,
 8.867368805193196e+17,
 8.866803364779336e+17,
 8.863661447344456e+17,
 8.862670092850176e+17,
 8.862583841518879e+17,
 8.860541600590725e+17,
 8.859848000199475e+17,
 8.855289432054702e+17,
 8.855189715287204e+17,
 8.853115929126093e+17,
 8.851676198836388e+17,
 8.849255217417093e+17,
 8.848767533904896e+17,
 8.845628921456886e+17,
 8.844418053827174e+17,
 8.8424787885149

### Append values

In [66]:
new_list.append(4)
new_list

[1, 5, 3, 4]

### Sort values

In [67]:
# temporarily
sorted(new_list)

[1, 3, 4, 5]

In [68]:
# permanently
new_list.sort()

In [69]:
# reverse
new_list.reverse()
new_list

[5, 4, 3, 1]

### Remove values

In [70]:
new_list.remove(1)

### Change values

In [71]:
new_list[1]='hello'
new_list

[5, 'hello', 3]

In [72]:
new_list[-2]='okay'
new_list

[5, 'okay', 3]

In [73]:
new_list[1]=3
new_list

[5, 3, 3]

## Dictonary

In [74]:
new_dict = {'joni': 31, 'anne': 33, 'zwörgü':0.5}

### Access values

In [75]:
for keys, values in new_dict_order.items(): 
    print (keys, values)

NameError: name 'new_dict_order' is not defined

In [76]:
new_dict.keys()

dict_keys(['joni', 'anne', 'zwörgü'])

In [77]:
new_dict.values()

dict_values([31, 33, 0.5])

In [78]:
new_dict.keys()

dict_keys(['joni', 'anne', 'zwörgü'])

In [79]:
new_dict.items()

dict_items([('joni', 31), ('anne', 33), ('zwörgü', 0.5)])

In [80]:
# access values for key
new_dict_details.get('anne')

NameError: name 'new_dict_details' is not defined

In [81]:
new_dict_details['anne']

NameError: name 'new_dict_details' is not defined

### Append values

In [82]:
mini_zwörgü = 0.1
new_dict['mini_zwörgü'] = mini_zwörgü
new_dict

{'joni': 31, 'anne': 33, 'zwörgü': 0.5, 'mini_zwörgü': 0.1}

In [83]:
mini_zwörgü = {'age': 0.1, 'height':0.1}
new_dict_details['mini_zwörgü'] = mini_zwörgü
new_dict_details

NameError: name 'new_dict_details' is not defined

### Sort values

In [84]:
import collections

new_dict_order = collections.OrderedDict(sorted(new_dict.items()))
new_dict_order

OrderedDict([('anne', 33),
             ('joni', 31),
             ('mini_zwörgü', 0.1),
             ('zwörgü', 0.5)])

In [85]:
new_dict_details = {'joni': {'age': 31, 'height':185}, 'anne': {'age': 33, 'height':165}, 'zwörgü': {'age':0.5, 'height':0.5}}
new_dict_details

{'joni': {'age': 31, 'height': 185},
 'anne': {'age': 33, 'height': 165},
 'zwörgü': {'age': 0.5, 'height': 0.5}}

## Loops

#### For loops

In [86]:
long_columns = []

for column in columns:
    length = len(column)
    if length > 10:
        long_columns.append(column)

long_columns

['in_reply_to_status_id',
 'in_reply_to_user_id',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator']

In [87]:
id_columns = []

for column in columns:
    if 'id' in column:
        id_columns.append(column)  

id_columns

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'retweeted_status_id',
 'retweeted_status_user_id']

In [88]:
meet_list = []

for i in range(0, len(tae)):
    if 'meet' in tae.test.iloc[i]:
        meet = tae[['tweet_id', 'test']].iloc[i]
        meet_list.append(meet)

meet_list

[tweet_id                                 886680336477933568.0
 test        This is Derek. He's late for a dog meeting. 13...
 Name: 27, dtype: object,
 tweet_id                                 884162670584377344.0
 test        Meet Yogi. He doesn't have any important dog m...
 Name: 42, dtype: object,
 tweet_id                                 779123168116150272.0
 test        This is Reggie. He hugs everyone he meets. 12/...
 Name: 749, dtype: object,
 tweet_id                                 669353438988365824.0
 test        This is Tessa. She is also very pleased after ...
 Name: 2168, dtype: object,
 tweet_id                                 667806454573760512.0
 test        This is Filup. He is overcome with joy after f...
 Name: 2250, dtype: object]

In [89]:
meet_df = pd.DataFrame(meet_list, columns={'tweet_id', 'test'})

In [90]:
meet_df.index

Int64Index([27, 42, 749, 2168, 2250], dtype='int64')

In [91]:
text_list = []

for i in range (0, len(meet_df.index)):
    x = meet_df.index[i]
    text_list.append(tae.test.iloc[x])

In [92]:
for i in range(0, len(tae)):
    if 'meat' in tae.test.iloc[i]:
        print(tae.test.iloc[i])

"So... we meat again" (I'm so sorry for that pun I couldn't resist pls don't unfollow) 10/10 https://t.co/XFBrrqapZa


In [93]:
test_list = []

for i in range(0, len(tae)):
    if 'meet' in (tae.test.iloc[i]):
        test_list.append(tae.test.iloc[i])

test_list

["This is Derek. He's late for a dog meeting. 13/10 pet...al to the metal https://t.co/BCoWue0abA",
 "Meet Yogi. He doesn't have any important dog meetings today he just enjoys looking his best at all times. 12/10 for dangerously dapper doggo https://t.co/YSI00BzTBZ",
 'This is Reggie. He hugs everyone he meets. 12/10 keep spreading the love Reggie https://t.co/uMfhduaate',
 'This is Tessa. She is also very pleased after finally meeting her biological father. 10/10 https://t.co/qDS1aCqppv',
 'This is Filup. He is overcome with joy after finally meeting his father. 10/10 https://t.co/TBmDJXJB75']

In [94]:
# delete certain text in rows
for i in range(0, len(tae)):
    tae.timestamp[i] = tae.timestamp[i][11:19]

<ipython-input-94-2f54d37b0fd3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tae.timestamp[i] = tae.timestamp[i][11:19]


KeyError: 1

#### If statements

In [95]:
'9' in tae.tweet_id[0].astype(str)

True

In [96]:
tweet_id_list_8 = []
tweet_id_list_9 = []

for i in range(0, len(tae)):
    if '9' in tae.tweet_id[i].astype(str):
        tweet_id_list_9.append(tae.tweet_id[i])
    elif '8' in tae.tweet_id[i].astype(str):
        tweet_id_list_8.append(tae.tweet_id[i])
    else:
        print('Unrecognizeable')

KeyError: 1

In [97]:
text_length = []

for i in range (0, len(tae)):
    x = len(tae.test[i].split(' '))
    text_length.append(x)

text_length

KeyError: 1

In [ ]:
# count words in tae.text - dict
text_length_dict = {}

for i in range (0, len(tae)):
    x = len(tae.text[i].split(' '))   
    new_variable = x
    text_length_dict[i] = new_variable

text_length_dict

In [98]:
# count words in tae.text - dataframe
text_length_df = pd.DataFrame(columns=['number', 'word_count'])

for i in range (0, len(tae)):
    x = len(tae.text[i].split(' '))
    df_append = pd.DataFrame([[i, x]], columns = ['number', 'word_count'])
    text_length_df = text_length_df.append(df_append)

text_length_df

AttributeError: 'DataFrame' object has no attribute 'text'

### Strings

In [99]:
for i in range(0, len(tae)):
    tae.test[i] = tae.test[i].lower()

<ipython-input-99-fd1015c6fe6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tae.test[i] = tae.test[i].lower()


KeyError: 1

In [100]:
for i in range(0, len(tae)):
    tae.test[i] = tae.test[i].upper()

<ipython-input-100-dbab00879948>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tae.test[i] = tae.test[i].upper()


KeyError: 1

In [ ]:
for i in range(0, len(tae)):
    tae.test[i] = tae.test[i].title()

In [101]:
tae.filter(regex='id$') # how it ends (columns)
tae.filter(regex='^r') # how it begins (columns)
tae.filter(regex='^r[1-5]$') # how it begins r and ends between 1-5 (columns)
tae.filter(regex='^(?!Species$).*') # except for species

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,test,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
0,8.924206e+17,NaN,NaN,16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",THIS IS PHINEAS. HE'S A MYSTICAL BOY. ONLY EVE...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,,,,,23
hallo,8.921774e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,,,,,23
2,8.918152e+17,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,,,,,22
3,8.916896e+17,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,,,,,23
4,8.913276e+17,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,,,,,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,6.660492e+17,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,,,,,15
2351,6.660442e+17,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,,,,,16
2352,6.660334e+17,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,,,,,19
2353,6.660293e+17,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,,,,,17


In [102]:
tae.test_new = tae.test.str.replace('meet', 'meat')

<ipython-input-102-fdfdac1b5277>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tae.test_new = tae.test.str.replace('meet', 'meat')


## Functions

In [103]:
def anne_function (input_1, input_2):
    anne_output = input_1 * input_2
    return anne_output

In [104]:
anne_function(3,4)

12

In [105]:
zwörgü_function = lambda input_1, input_2: input_1 + input_2

In [106]:
zwörgü_function(3,4)

7